## PCA on $(r, n_1, n_2)$:

The dimensions for each simulation is (44, 21, 21).

In [1]:
44*21*21/2

9702.0

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from gal_goku import summary_stats
from gal_goku import plot

fid = 'HF'

data_dir = '/home/qezlou/HD2/HETDEX/cosmo/data/xi_on_grid/'
xi = summary_stats.Xi(data_dir, fid, logging_level='ERROR')
plot_xi = plot.PlotXiSims()
rbins, mass_pairs, corr_2d = xi.load_all()

print(corr_2d.shape, mass_pairs.shape)

(21, 231, 50) (231, 2)


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from gal_goku import summary_stats
from gal_goku import plot

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def do_pca_n1_n2(n_components=10, fid='HF'):
    """
    Check the accuracy of the PCA approximation for the `xi(r, n1, n2)` correlation
    function with `n_components` number of components.
    """
    ## Load the data
    plot_xi = plot.PlotXiSims() 
    data_dir = '/home/qezlou/HD2/HETDEX/cosmo/data/xi_on_grid/'
    xi = summary_stats.Xi(data_dir, fid, logging_level='ERROR')
    ## Load a 2D array, (sim, mass_pairs, xi(r))
    _, mass_pairs, corr = xi.load_all()
    orig_shape = corr.shape
    print(orig_shape )

    per_panel = 1
    rows, columns = plot_xi._setup_panels(sim_nums=len(xi.sim_tags), per_panel=per_panel)
    fig, ax = plt.subplots(rows, columns, figsize=(4*columns, 3*rows))
    for i in range(orig_shape[0]):
        corrs = corr.reshape(corr.shape[0], -1)
        scaler = StandardScaler()
        corrs_transformed = scaler.fit_transform(np.delete(corrs, i, axis=0))
        print(f'training on {corrs_transformed.shape}')

        pca = PCA(n_components=n_components)
        pca.fit(corrs_transformed)

        # Compare the original and approximated correlation functions
        corrs_approx = pca.inverse_transform(pca.transform(corrs[i][np.newaxis, :]))
        corrs_approx = scaler.inverse_transform(corrs_approx)

        corrs_approx = corrs_approx.reshape(orig_shape[1], orig_shape[2])
    
   

        indx, indy = np.floor(i/columns).astype(int), i%columns
        im = ax[indx, indy].imshow(corrs_approx/corr[i] - 1, origin='lower', 
                                    extent=[mass_bins[0], mass_bins[-1], mass_bins[0],  mass_bins[-1],], 
                                    cmap='coolwarm', aspect='auto', vmin=-0.1, vmax=0.1)
        cbar = fig.colorbar(im, ax=ax[indx, indy], fraction=0.046, pad=0.04)
    fig.suptitle(f'LOO {n_components} PCA components vs original {orig_shape[1]*orig_shape[2]} bins | fid {fid}')
    fig.tight_layout()
#do_pca_n1_n2(n_components=10)
do_pca_n1_n2(n_components=15, fid='HF')



(564, 231, 50)
training on (563, 11550)


ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values